# Label Propagation on email dataset

In [42]:
!scalac --version

Scala compiler version 2.13.5 -- Copyright 2002-2020, LAMP/EPFL and Lightbend, Inc.



In [43]:
import scala.util.Random
import org.apache.spark.graphx.EdgeTriplet
import org.apache.spark.graphx.Graph
import org.apache.spark.graphx.VertexId
import org.apache.spark.sql.expressions.Window
import org.apache.spark.rdd.RDD
import org.apache.spark.graphx.EdgeDirection
import org.apache.spark.graphx.Edge
import org.apache.spark.sql.Row
import org.apache.spark.graphx.GraphLoader
import org.apache.spark.sql.functions.rand

import scala.util.Random
import org.apache.spark.graphx.EdgeTriplet
import org.apache.spark.graphx.Graph
import org.apache.spark.graphx.VertexId
import org.apache.spark.sql.expressions.Window
import org.apache.spark.rdd.RDD
import org.apache.spark.graphx.EdgeDirection
import org.apache.spark.graphx.Edge
import org.apache.spark.sql.Row
import org.apache.spark.graphx.GraphLoader
import org.apache.spark.sql.functions.rand


In [44]:
// Defining Vertex class in order to take into account labels
class LPVertex(
    var degree : Int = 0,
    var initialLabel : Boolean = false,
    // string label name is associated with its probability
    var vertexLabels : Map[String, Double] = Map[String, Double]() 
) extends java.io.Serializable {     
    
    def setDegree(degree: Int) = {
        this.degree = degree
    }
    
   // Overriding tostring method
    override def toString() : String = {
    return "initialLabel : " + initialLabel + "/ vertexLabels = " + 
        vertexLabels
    }
}

defined class LPVertex


In [45]:
// Vertex Progam for Pregel
def VertexProgram(id : VertexId, 
                  VD : LPVertex, 
                  m : Map[String, Double]) : LPVertex  = {
    var newLPVertex = new LPVertex()
    newLPVertex.degree = VD.degree
    newLPVertex.initialLabel = VD.initialLabel
    newLPVertex.vertexLabels = VD.vertexLabels
    
    if(VD.initialLabel == false){
        newLPVertex.vertexLabels = m.map(x => (x._1, x._2 / VD.degree))
    }
    newLPVertex
}

VertexProgram: (id: org.apache.spark.graphx.VertexId, VD: LPVertex, m: Map[String,Double])LPVertex


In [46]:
// Send Message
def SendMsg(triplet : EdgeTriplet[LPVertex, Int]): Iterator[(VertexId, Map[String, Double])] = {
    Iterator((triplet.dstId, triplet.srcAttr.vertexLabels))
}

SendMsg: (triplet: org.apache.spark.graphx.EdgeTriplet[LPVertex,Int])Iterator[(org.apache.spark.graphx.VertexId, Map[String,Double])]


In [47]:
// Merge Message
def MergeMsg(m1 : Map[String, Double],
             m2 : Map[String, Double]) : Map[String, Double] = {
    val mergeMsg = (m1.keySet ++ m2.keySet).map {i=> (i,m1.getOrElse(i,0.0) + m2.getOrElse(i,0.0))}.toMap
    mergeMsg;
}

MergeMsg: (m1: Map[String,Double], m2: Map[String,Double])Map[String,Double]


### Loading data

### Getting labeled data

In [48]:
// loading community labels
val email_labels = sc.textFile("email/email-Eu-core-department-labels.txt")

email_labels: org.apache.spark.rdd.RDD[String] = email/email-Eu-core-department-labels.txt MapPartitionsRDD[1868] at textFile at <console>:49


In [49]:
// creating dataframe from it
var email_label_data = email_labels.toDF.withColumn("id",monotonicallyIncreasingId)

email_label_data = email_label_data.withColumn("rank", row_number().over(Window.orderBy("id")))

val email_label_splitted = email_label_data.withColumn("_tmp", split($"value", "\\ ")).select(
  $"_tmp".getItem(0).as("email"),
  $"_tmp".getItem(1).as("community"),
)

email_label_data: org.apache.spark.sql.DataFrame = [value: string, id: bigint ... 1 more field]
email_label_data: org.apache.spark.sql.DataFrame = [value: string, id: bigint ... 1 more field]
email_label_splitted: org.apache.spark.sql.DataFrame = [email: string, community: string]


In [50]:
val shuffled = email_label_splitted.orderBy()
val all_labels = shuffled.rdd.map(x => (x.getString(0).toInt, x.getString(1).toInt))
val all_labels2 = all_labels.collectAsMap()

shuffled: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [email: string, community: string]
all_labels: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[1874] at map at <console>:55
all_labels2: scala.collection.Map[Int,Int] = Map(137 -> 21, 891 -> 38, 864 -> 28, 146 -> 15, 550 -> 15, 559 -> 27, 218 -> 1, 568 -> 4, 227 -> 1, 765 -> 14, 774 -> 21, 92 -> 9, 433 -> 37, 846 -> 1, 101 -> 16, 442 -> 10, 855 -> 15, 200 -> 4, 173 -> 38, 514 -> 8, 209 -> 19, 523 -> 4, 182 -> 7, 254 -> 40, 191 -> 0, 720 -> 14, 729 -> 14, 828 -> 38, 424 -> 34, 738 -> 22, 83 -> 36, 837 -> 6, 810 -> 26, 155 -> 35, 496 -> 14, 469 -> 22, 819 -> 17, 164 -> 17, 478 -> 8, 505 -> 14, 577 -> 27, 245 -> 10, 783 -> 23, 792 -> 24, 110 -> 8, 451 -> 14, 801 -> 4, 119 -> 10, 460 -> 34, 873 -> 15, 532 -> 21, 128 ...


In [51]:
// getting only one element from each community
def get_unique_labels(g:org.apache.spark.sql.Dataset[Row]) = {
    val shuffled = g.orderBy(rand())
    val unique = shuffled.dropDuplicates("community")
    val label_map = unique.rdd.map(x => (x.getString(0).toInt, x.getString(1).toInt))
    label_map.collectAsMap()
}

get_unique_labels: (g: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row])scala.collection.Map[Int,Int]


In [52]:
// getting a subset of nodes
def get_subset_label(g:org.apache.spark.sql.Dataset[Row], nb_samples:Int) = {
    val shuffled = g.orderBy(rand())
    val unique = shuffled.limit(nb_samples)
    val label_map = unique.rdd.map(x => (x.getString(0).toInt, x.getString(1).toInt))
    label_map.collectAsMap()
}

get_subset_label: (g: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row], nb_samples: Int)scala.collection.Map[Int,Int]


In [53]:
val label_map : Map[Int,Int] = get_unique_labels(email_label_splitted).toMap
val label_map2 : Map[Int,Int] = get_unique_labels(email_label_splitted).toMap
val label_map3 : Map[Int,Int] = get_unique_labels(email_label_splitted).toMap
val subset_label_map : Map[Int,Int] = get_subset_label(email_label_splitted, 200).toMap

label_map: Map[Int,Int] = Map(941 -> 41, 408 -> 21, 730 -> 23, 614 -> 24, 861 -> 5, 870 -> 33, 152 -> 37, 492 -> 40, 623 -> 4, 646 -> 22, 834 -> 6, 353 -> 16, 480 -> 15, 132 -> 28, 307 -> 1, 233 -> 19, 810 -> 26, 862 -> 2, 28 -> 11, 989 -> 8, 96 -> 20, 530 -> 14, 819 -> 17, 445 -> 3, 181 -> 7, 113 -> 10, 535 -> 9, 816 -> 13, 769 -> 27, 278 -> 0, 80 -> 29, 145 -> 35, 175 -> 32, 876 -> 30, 87 -> 34, 891 -> 38, 268 -> 39, 767 -> 18, 427 -> 12, 528 -> 25, 83 -> 36, 560 -> 31)


In [121]:
val aux = label_map ++ label_map2 
val merged_label_map = aux.toMap

label_map2: Map[Int,Int] = Map(941 -> 41, 352 -> 0, 170 -> 13, 582 -> 15, 217 -> 37, 870 -> 33, 492 -> 40, 591 -> 23, 106 -> 38, 966 -> 29, 821 -> 24, 89 -> 20, 988 -> 22, 206 -> 4, 60 -> 7, 810 -> 26, 329 -> 8, 980 -> 12, 593 -> 5, 901 -> 21, 642 -> 6, 590 -> 11, 558 -> 14, 445 -> 3, 784 -> 28, 434 -> 34, 494 -> 36, 631 -> 19, 140 -> 17, 726 -> 31, 521 -> 2, 768 -> 16, 812 -> 32, 331 -> 39, 154 -> 35, 876 -> 30, 171 -> 10, 603 -> 1, 767 -> 18, 400 -> 9, 528 -> 25, 454 -> 27)


In [124]:
val aux2 = merged_label_map ++ label_map3
val merged_label_map2 = aux2.toMap

aux2: scala.collection.immutable.Map[Int,Int] = Map(809 -> 22, 941 -> 41, 777 -> 9, 88 -> 20, 352 -> 0, 408 -> 21, 170 -> 13, 582 -> 15, 730 -> 23, 217 -> 37, 614 -> 24, 861 -> 5, 814 -> 0, 340 -> 2, 288 -> 37, 702 -> 4, 357 -> 8, 853 -> 7, 870 -> 33, 152 -> 37, 492 -> 40, 756 -> 39, 591 -> 23, 623 -> 4, 646 -> 22, 834 -> 6, 106 -> 38, 353 -> 16, 966 -> 29, 821 -> 24, 480 -> 15, 132 -> 28, 89 -> 20, 660 -> 17, 988 -> 22, 206 -> 4, 307 -> 1, 233 -> 19, 60 -> 7, 85 -> 1, 810 -> 26, 862 -> 2, 28 -> 11, 820 -> 5, 329 -> 8, 842 -> 16, 615 -> 35, 980 -> 12, 593 -> 5, 420 -> 26, 989 -> 8, 901 -> 21, 642 -> 6, 590 -> 11, 212 -> 34, 96 -> 20, 541 -> 23, 558 -> 14, 530 -> 14, 64 -> 25, 819 -> 17, 44 -> 14, 445 -> 3, 144 -> 40, 181 -> 7, 900 -> 13, 113 -> 10, 784 -> 28, 76 -> 10, 434 -> 34, 494 ->...


In [127]:
//merging merged_label_map2 and subset_label_map
val aux3 = merged_label_map2 ++ subset_label_map
val last_merge = aux3.toMap

aux3: scala.collection.immutable.Map[Int,Int] = Map(892 -> 38, 809 -> 22, 760 -> 17, 347 -> 0, 846 -> 1, 628 -> 7, 249 -> 14, 941 -> 41, 777 -> 9, 555 -> 14, 88 -> 20, 352 -> 0, 408 -> 21, 170 -> 13, 582 -> 15, 762 -> 21, 115 -> 13, 730 -> 23, 217 -> 37, 741 -> 7, 449 -> 1, 878 -> 11, 614 -> 24, 677 -> 26, 861 -> 5, 533 -> 35, 500 -> 14, 715 -> 15, 814 -> 0, 913 -> 14, 340 -> 2, 829 -> 4, 404 -> 16, 782 -> 6, 288 -> 37, 184 -> 15, 372 -> 36, 110 -> 8, 587 -> 21, 619 -> 6, 702 -> 4, 357 -> 8, 817 -> 4, 853 -> 7, 20 -> 9, 870 -> 33, 46 -> 17, 152 -> 37, 289 -> 0, 316 -> 1, 475 -> 8, 492 -> 40, 756 -> 39, 789 -> 4, 591 -> 23, 321 -> 7, 623 -> 4, 211 -> 22, 646 -> 22, 834 -> 6, 106 -> 38, 353 -> 16, 966 -> 29, 821 -> 24, 480 -> 15, 981 -> 10, 280 -> 4, 634 -> 21, 877 -> 15, 453 -> 6, 132 ->...


In [128]:
val x = (0 to 41).toArray

x: Array[Int] = Array(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41)


In [129]:
// Defining function to assign labeled nodes
def transform(VD: VertexId): LPVertex = {
    val x = (0 to 41).toArray
    val emptyLabels : Map[String,Double] = x.groupBy(identity).map{case (x,y) => x.toString -> 0.0}
    var v = new LPVertex(0, false, emptyLabels);
    if (label_map.contains(VD.toInt)) {
        val init = emptyLabels ++ Map(label_map(VD.toInt).toString -> 1.0)

        v.initialLabel = true;
        v.vertexLabels = init;
    }
    v;
}

transform: (VD: org.apache.spark.graphx.VertexId)LPVertex


In [130]:
// Defining function to assign labeled nodes
def transform2(VD: VertexId): LPVertex = {
    val x = (0 to 41).toArray
    val emptyLabels : Map[String,Double] = x.groupBy(identity).map{case (x,y) => x.toString -> 0.0}
    var v = new LPVertex(0, false, emptyLabels);
    if (subset_label_map.contains(VD.toInt)) {
        val init = emptyLabels ++ Map(subset_label_map(VD.toInt).toString -> 1.0)

        v.initialLabel = true;
        v.vertexLabels = init;
    }
    v;
}

transform2: (VD: org.apache.spark.graphx.VertexId)LPVertex


In [131]:
def transform3(VD: VertexId): LPVertex = {
    val x = (0 to 41).toArray
    val emptyLabels : Map[String,Double] = x.groupBy(identity).map{case (x,y) => x.toString -> 0.0}
    var v = new LPVertex(0, false, emptyLabels);
    if (merged_label_map.contains(VD.toInt)) {
        val init = emptyLabels ++ Map(merged_label_map(VD.toInt).toString -> 1.0)

        v.initialLabel = true;
        v.vertexLabels = init;
    }
    v;
}

transform3: (VD: org.apache.spark.graphx.VertexId)LPVertex


In [132]:
def transform4(VD: VertexId): LPVertex = {
    val x = (0 to 41).toArray
    val emptyLabels : Map[String,Double] = x.groupBy(identity).map{case (x,y) => x.toString -> 0.0}
    var v = new LPVertex(0, false, emptyLabels);
    if (last_merge.contains(VD.toInt)) {
        val init = emptyLabels ++ Map(last_merge(VD.toInt).toString -> 1.0)

        v.initialLabel = true;
        v.vertexLabels = init;
    }
    v;
}

transform4: (VD: org.apache.spark.graphx.VertexId)LPVertex


## Merging all the data together

In [133]:
// Loading graph
val initialGraph = GraphLoader.edgeListFile(sc, "email/email-Eu-core.txt", canonicalOrientation=false)

initialGraph: org.apache.spark.graphx.Graph[Int,Int] = org.apache.spark.graphx.impl.GraphImpl@3613187b


In [134]:
// val graph = initialGraph.mapVertices({case (id, attr) => transform(id)})
// val graph = initialGraph.mapVertices({case (id, attr) => transform2(id)})
// val graph = initialGraph.mapVertices({case (id, attr) => transform3(id)})
val graph = initialGraph.mapVertices({case (id, attr) => transform4(id)})

val vertex_rdd = graph.degrees.zip(graph.vertices).map({ case (vDeg, vAttr) => vAttr._2.setDegree(vDeg._2) ; vAttr})
val edge_rdd = graph.edges

val graph2 = Graph(vertex_rdd, edge_rdd)

// We have an undirected graph
val finalGraph = Graph(graph2.vertices, graph2.edges.union(graph2.edges.reverse))

graph: org.apache.spark.graphx.Graph[LPVertex,Int] = org.apache.spark.graphx.impl.GraphImpl@61189ebc
vertex_rdd: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, LPVertex)] = MapPartitionsRDD[7321] at map at <console>:62
edge_rdd: org.apache.spark.graphx.EdgeRDD[Int] = EdgeRDDImpl[7315] at RDD at EdgeRDD.scala:41
graph2: org.apache.spark.graphx.Graph[LPVertex,Int] = org.apache.spark.graphx.impl.GraphImpl@2134cf88
finalGraph: org.apache.spark.graphx.Graph[LPVertex,Int] = org.apache.spark.graphx.impl.GraphImpl@77b46403


In [135]:
graph.vertices.collect()

res38: Array[(org.apache.spark.graphx.VertexId, LPVertex)] = Array((384,initialLabel : false/ vertexLabels = Map(34 -> 0.0, 12 -> 0.0, 8 -> 0.0, 19 -> 0.0, 23 -> 0.0, 4 -> 0.0, 40 -> 0.0, 15 -> 0.0, 11 -> 0.0, 9 -> 0.0, 33 -> 0.0, 22 -> 0.0, 26 -> 0.0, 37 -> 0.0, 13 -> 0.0, 24 -> 0.0, 35 -> 0.0, 16 -> 0.0, 5 -> 0.0, 10 -> 0.0, 21 -> 0.0, 32 -> 0.0, 6 -> 0.0, 36 -> 0.0, 1 -> 0.0, 39 -> 0.0, 17 -> 0.0, 25 -> 0.0, 14 -> 0.0, 31 -> 0.0, 0 -> 0.0, 20 -> 0.0, 27 -> 0.0, 2 -> 0.0, 38 -> 0.0, 18 -> 0.0, 30 -> 0.0, 7 -> 0.0, 29 -> 0.0, 41 -> 0.0, 3 -> 0.0, 28 -> 0.0)), (454,initialLabel : true/ vertexLabels = Map(34 -> 0.0, 12 -> 0.0, 8 -> 0.0, 19 -> 0.0, 23 -> 0.0, 4 -> 0.0, 40 -> 0.0, 15 -> 0.0, 11 -> 0.0, 9 -> 0.0, 33 -> 0.0, 22 -> 0.0, 26 -> 0.0, 37 -> 0.0, 13 -> 0.0, 24 -> 0.0, 35 -> 0.0, 1...


In [136]:
val initialMsg = x.groupBy(identity).map{case (x,y) => x.toString -> 0.0}
val maxIter = 100

initialMsg: scala.collection.immutable.Map[String,Double] = Map(34 -> 0.0, 12 -> 0.0, 8 -> 0.0, 19 -> 0.0, 23 -> 0.0, 4 -> 0.0, 40 -> 0.0, 15 -> 0.0, 11 -> 0.0, 9 -> 0.0, 33 -> 0.0, 22 -> 0.0, 26 -> 0.0, 37 -> 0.0, 13 -> 0.0, 24 -> 0.0, 35 -> 0.0, 16 -> 0.0, 5 -> 0.0, 10 -> 0.0, 21 -> 0.0, 32 -> 0.0, 6 -> 0.0, 36 -> 0.0, 1 -> 0.0, 39 -> 0.0, 17 -> 0.0, 25 -> 0.0, 14 -> 0.0, 31 -> 0.0, 0 -> 0.0, 20 -> 0.0, 27 -> 0.0, 2 -> 0.0, 38 -> 0.0, 18 -> 0.0, 30 -> 0.0, 7 -> 0.0, 29 -> 0.0, 41 -> 0.0, 3 -> 0.0, 28 -> 0.0)
maxIter: Int = 100


In [137]:
finalGraph.vertices.foreach(v => println(v._1, v._2.degree, v._2.initialLabel, v._2.vertexLabels.mkString(" ")))

(451,20,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(893,2,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(147,82,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 

(755,1,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(605,4,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(659,3,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13

(547,49,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(855,7,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(701,1,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 1

(275,57,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(541,31,true,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 1.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(427,94,true,34 -> 0.0 12 -> 1.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 1

(987,12,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(869,9,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(259,50,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 

...

(756,41,true,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 1.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(708,29,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(979,7,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 1

(440,127,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(876,1,true,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 1.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(526,95,true,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 1.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 1

(614,6,true,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 1.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(424,203,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(666,35,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 

(638,29,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(648,2,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(610,24,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 

(642,28,true,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 1.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(468,21,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(448,69,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 

In [138]:
val graphLP = finalGraph.pregel(initialMsg, maxIter, EdgeDirection.Either)(
    (id, vert, newLabels) => VertexProgram(id, vert, newLabels),
    it => SendMsg(it),
    (a, b) => MergeMsg(a, b)
)

graphLP: org.apache.spark.graphx.Graph[LPVertex,Int] = org.apache.spark.graphx.impl.GraphImpl@1c774246


In [139]:
graphLP.vertices.foreach(v => println(v._1, v._2.degree, v._2.initialLabel, v._2.vertexLabels.mkString(" ")))

(451,20,false,34 -> 0.024546123377374635 12 -> 0.0018656280496002389 8 -> 0.01058538570500211 19 -> 0.012033736402916707 23 -> 0.003296395434791377 4 -> 0.029288351392140876 40 -> 0.0013208460700823987 15 -> 0.01676393415792344 11 -> 0.006401759239979687 9 -> 0.012604654343339383 33 -> 6.516695473206721E-5 22 -> 0.009941557957915416 26 -> 0.013767986621177291 37 -> 0.0048111045198048555 13 -> 0.014198068144084124 24 -> 1.526187730279422E-4 35 -> 0.02229833513229301 16 -> 0.00567718717834591 5 -> 0.009261685869766672 10 -> 0.007097402041368717 21 -> 0.011944027211063812 32 -> 0.003691990717031977 6 -> 0.0032849354252866073 36 -> 0.03419335265764224 1 -> 0.01375762375247297 39 -> 0.008605122388287433 17 -> 0.008386856918994436 25 -> 0.018905995908616198 14 -> 0.617705617935649 31 -> 0.01187267585368774 0 -> 0.014158256222135849 20 -> 0.0062890877857738016 27 -> 0.003297828769855867 2 -> 0.005598632036053056 38 -> 0.006880038751460722 18 -> 1.4264548630868598E-4 30 -> 9.455166076869604E-5

(215,148,false,34 -> 0.06134399393087835 12 -> 0.003558530056500435 8 -> 0.018162328222543524 19 -> 0.025465487195913634 23 -> 0.019244911397186565 4 -> 0.05262806454466145 40 -> 0.0020434759161221364 15 -> 0.04655078946648432 11 -> 0.009502792676489586 9 -> 0.04620385669203479 33 -> 7.681492735212726E-5 22 -> 0.018223100014668814 26 -> 0.010757839131603483 37 -> 0.008822129347317879 13 -> 0.021009870982678876 24 -> 0.001415530475186163 35 -> 0.038522187251856466 16 -> 0.007193104328797222 5 -> 0.029352410475473795 10 -> 0.012473169104035649 21 -> 0.022829376641942414 32 -> 0.0198215006591319 6 -> 0.007458383817346845 36 -> 0.051307756537300964 1 -> 0.11594308301654062 39 -> 0.05328540128219971 17 -> 0.009544464764870145 25 -> 0.024969301647298523 14 -> 0.05098737933615163 31 -> 0.025164019390135094 0 -> 0.03214631807743655 20 -> 0.01735407937766302 27 -> 0.004366296950130742 2 -> 0.014202328917355343 38 -> 0.01942291204651373 18 -> 3.0347479973769667E-4 30 -> 1.0148558539099477E-4 7 -

(577,18,false,34 -> 0.04140720194829317 12 -> 0.026726488445964175 8 -> 0.03192738673810293 19 -> 0.017905946413052314 23 -> 0.0039091793493653305 4 -> 0.05115076235624352 40 -> 0.00591258355410308 15 -> 0.028188201575329246 11 -> 0.014814594627347917 9 -> 0.022310086989339402 33 -> 6.987008796993568E-5 22 -> 0.019198250262478962 26 -> 0.010011132178890917 37 -> 0.015933697593583644 13 -> 0.027788133287774703 24 -> 2.1241668121512204E-4 35 -> 0.03596862906112871 16 -> 0.01240337768513981 5 -> 0.01495035156421008 10 -> 0.023869185052638058 21 -> 0.0221234936273306 32 -> 0.009396005382376373 6 -> 0.009087783005627412 36 -> 0.17535415544957253 1 -> 0.0250593173439778 39 -> 0.006664665391544192 17 -> 0.012544470013642738 25 -> 0.01517700301390947 14 -> 0.16011351228639845 31 -> 0.008976336959274918 0 -> 0.02844268471314309 20 -> 0.01634097264251205 27 -> 0.00958446843029571 2 -> 0.010095130618782136 38 -> 0.017019415618530813 18 -> 2.3602897309272128E-4 30 -> 8.18530565797993E-5 7 -> 0.033

(489,59,false,34 -> 0.09961122812733882 12 -> 0.0043159413685793764 8 -> 0.01795584483143706 19 -> 0.018639407127902297 23 -> 0.004608299483020947 4 -> 0.07532079991991444 40 -> 0.002406340137293606 15 -> 0.02102416669690343 11 -> 0.012373182141874299 9 -> 0.050081152997924705 33 -> 5.688544045528277E-5 22 -> 0.05291377097278831 26 -> 0.008698910829999695 37 -> 0.006585684811938755 13 -> 0.027703743176742887 24 -> 1.3706130983135074E-4 35 -> 0.020337524113591392 16 -> 0.01043731602295586 5 -> 0.011218144839746605 10 -> 0.012158917802977164 21 -> 0.07162799649695731 32 -> 0.009568485436549753 6 -> 0.0078450840770852 36 -> 0.035099168274828355 1 -> 0.01550461770595346 39 -> 0.004059116440627968 17 -> 0.02987595743518643 25 -> 0.13897303158919472 14 -> 0.06813353606014198 31 -> 0.004169664737577797 0 -> 0.024790001900750806 20 -> 0.013979929439352591 27 -> 0.006535488950338702 2 -> 0.007471587881832175 38 -> 0.034685545727015314 18 -> 6.743493492431515E-4 30 -> 1.4332058670766512E-4 7 -> 

(363,83,false,34 -> 0.09774665742284422 12 -> 0.004389152716573508 8 -> 0.0414353687650759 19 -> 0.03940705363408215 23 -> 0.0032738262420686707 4 -> 0.0805171538315986 40 -> 0.0031236382968082054 15 -> 0.023308772703117986 11 -> 0.03510830036142575 9 -> 0.03321235611075119 33 -> 9.068470322051946E-5 22 -> 0.03889388825021967 26 -> 0.023056500717845443 37 -> 0.022666674946670188 13 -> 0.03835568846015345 24 -> 1.5921160106464503E-4 35 -> 0.07300658622154806 16 -> 0.009297400732264881 5 -> 0.010681818898993682 10 -> 0.01273051194470172 21 -> 0.046418486077624596 32 -> 0.0065114488863160435 6 -> 0.006874896210378051 36 -> 0.03834923566846625 1 -> 0.02295578569297911 39 -> 0.004585954074711745 17 -> 0.013339419040691958 25 -> 0.01241848096894423 14 -> 0.07223377252331221 31 -> 0.01665513146976284 0 -> 0.06736005858940039 20 -> 0.012741238259030728 27 -> 0.006035766498168164 2 -> 0.008022575937483253 38 -> 0.026906695841813724 18 -> 3.4542878834502565E-4 30 -> 1.0327365083621774E-4 7 -> 0.

...

(537,61,false,34 -> 0.03152186801996895 12 -> 0.0030340728094498974 8 -> 0.014170060079340647 19 -> 0.016083326821073933 23 -> 0.008871365627048577 4 -> 0.09953126224436054 40 -> 0.0013404483077824408 15 -> 0.025298971924618083 11 -> 0.007697159320767288 9 -> 0.07399821281663169 33 -> 1.1513558231043415E-4 22 -> 0.010512426457015238 26 -> 0.009037458386605933 37 -> 0.011004780638850338 13 -> 0.056123323563805956 24 -> 0.002638802539206851 35 -> 0.021969978874751727 16 -> 0.0071016351074346465 5 -> 0.01879944794376733 10 -> 0.010643243369098344 21 -> 0.013571625974193608 32 -> 0.004924865817606454 6 -> 0.016321558760670193 36 -> 0.03622999098045997 1 -> 0.1129631545778873 39 -> 0.03222325939745485 17 -> 0.008783602676586032 25 -> 0.022367927910953097 14 -> 0.07717457947853859 31 -> 0.008253545672875678 0 -> 0.04570399112250328 20 -> 0.008229512083624229 27 -> 0.0041066861425091165 2 -> 0.006808185694702949 38 -> 0.03162691023434885 18 -> 3.9511754211978277E-4 30 -> 8.660894533697022E-5 

(610,24,false,34 -> 0.03325849828172509 12 -> 0.003296913050339048 8 -> 0.017134802647128983 19 -> 0.05260092348893455 23 -> 0.004706695032564536 4 -> 0.04710080265239861 40 -> 0.0017590259469784852 15 -> 0.020361512708444062 11 -> 0.008334132528805714 9 -> 0.015854988439163376 33 -> 7.542127758921208E-5 22 -> 0.03567709168168164 26 -> 0.06012260866069564 37 -> 0.00794044798502963 13 -> 0.018033745200936147 24 -> 4.712025765137552E-4 35 -> 0.021587348994186933 16 -> 0.009282341998552676 5 -> 0.009036388413795842 10 -> 0.010901058275201603 21 -> 0.09487766305325908 32 -> 0.030806290728047293 6 -> 0.020638689368498973 36 -> 0.04138128774377958 1 -> 0.018712578156275955 39 -> 0.0076370066182049655 17 -> 0.016404791500892385 25 -> 0.02914774048086777 14 -> 0.03597171821193723 31 -> 0.012460997052236397 0 -> 0.01972382835430841 20 -> 0.17907390866389705 27 -> 0.013423262609251654 2 -> 0.006073766192641432 38 -> 0.01303244870372478 18 -> 1.9472609689880378E-4 30 -> 1.250899649211123E-4 7 -> 

(358,88,true,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 1.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(746,2,false,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13 -> 0.0 24 -> 0.0 35 -> 0.0 16 -> 0.0 5 -> 0.0 10 -> 0.0 21 -> 0.0 32 -> 0.0 6 -> 0.0 36 -> 0.0 1 -> 0.0 39 -> 0.0 17 -> 0.0 25 -> 0.0 14 -> 0.0 31 -> 0.0 0 -> 0.0 20 -> 0.0 27 -> 0.0 2 -> 0.0 38 -> 0.0 18 -> 0.0 30 -> 0.0 7 -> 0.0 29 -> 0.0 41 -> 0.0 3 -> 0.0 28 -> 0.0)
(154,74,true,34 -> 0.0 12 -> 0.0 8 -> 0.0 19 -> 0.0 23 -> 0.0 4 -> 0.0 40 -> 0.0 15 -> 0.0 11 -> 0.0 9 -> 0.0 33 -> 0.0 22 -> 0.0 26 -> 0.0 37 -> 0.0 13

(724,2,false,34 -> 0.043266359819924834 12 -> 0.008705547512345941 8 -> 0.02796124888545904 19 -> 0.022454333908244067 23 -> 0.010679356874231959 4 -> 0.08072913870337023 40 -> 0.0043913706150345375 15 -> 0.04168456561163737 11 -> 0.03928369304513267 9 -> 0.02337181584668826 33 -> 8.832860527677987E-5 22 -> 0.0271395187784252 26 -> 0.014838772851056812 37 -> 0.010992823609274067 13 -> 0.1171584263344926 24 -> 0.002195176028105886 35 -> 0.02517739660221731 16 -> 0.014155401152393388 5 -> 0.015545315778619066 10 -> 0.050930546234846955 21 -> 0.0311445803635461 32 -> 0.011941387558590246 6 -> 0.011914202171135448 36 -> 0.04930073384160354 1 -> 0.02817452138128671 39 -> 0.006636000310534483 17 -> 0.023739229493457574 25 -> 0.020185544980893068 14 -> 0.050274031126416194 31 -> 0.008395539461407742 0 -> 0.03271832602346556 20 -> 0.03977579524371336 27 -> 0.010462172966085315 2 -> 0.010751962175336605 38 -> 0.016514544264280687 18 -> 3.425809042437969E-4 30 -> 2.7102291675678496E-4 7 -> 0.030

(776,44,false,34 -> 0.02818285464867709 12 -> 0.002825775149579034 8 -> 0.014143418895617016 19 -> 0.008135467486356653 23 -> 0.0034963641611518116 4 -> 0.2560745679544278 40 -> 0.001326533003951298 15 -> 0.01667683004845614 11 -> 0.011012787360472355 9 -> 0.014475854084504768 33 -> 4.0265554242103536E-4 22 -> 0.007376925651744693 26 -> 0.00794406593617797 37 -> 0.00502708172804885 13 -> 0.021350304010384593 24 -> 2.6281120692479154E-4 35 -> 0.020879927114941486 16 -> 0.00687137215577165 5 -> 0.2307111958927498 10 -> 0.0151603108577747 21 -> 0.012390196527971102 32 -> 0.0036863978072530098 6 -> 0.008308369488895039 36 -> 0.036283257543289336 1 -> 0.023357050793374082 39 -> 0.004037694062544258 17 -> 0.012156235098670204 25 -> 0.02548158040610502 14 -> 0.03242021096440202 31 -> 0.0039779324846718625 0 -> 0.05526758650081589 20 -> 0.008141250649998497 27 -> 0.003424720020036481 2 -> 0.007325341015734335 38 -> 0.0065324368085366265 18 -> 1.8934353841740993E-4 30 -> 5.7332139494110706E-5 7

In [140]:
val m = graphLP.vertices.map({case(k,v) => (k.toInt, v.vertexLabels.maxBy{ case (key, value) => value }._1.toInt)})

m: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[9065] at map at <console>:49


In [141]:
val m2 = m.collect()
val aux = m2.toMap
val predicted_labels = collection.mutable.Map(aux.toSeq: _*) 

m2: Array[(Int, Int)] = Array((454,27), (386,7), (522,4), (772,34), (324,14), (180,0), (320,7), (752,4), (586,21), (408,21), (428,17), (986,28), (996,14), (464,4), (346,0), (14,4), (466,14), (24,11), (520,21), (912,14), (302,20), (146,1), (140,17), (204,5), (514,8), (390,7), (226,1), (934,39), (318,10), (956,14), (12,14), (576,36), (810,26), (228,1), (628,7), (712,4), (452,14), (94,4), (450,4), (192,21), (160,4), (492,40), (512,35), (992,4), (954,35), (420,26), (366,36), (22,9), (460,13), (548,13), (350,0), (646,22), (808,34), (758,41), (842,16), (528,25), (696,1), (626,34), (40,11), (558,14), (434,34), (58,21), (590,11), (136,13), (944,4), (878,11), (196,4), (636,21), (96,20), (338,13), (784,28), (698,21), (524,9), (848,2), (962,15), (918,4), (216,17), (292,4), (98,16), (884,21), (30,1...


In [142]:
val merged = (predicted_labels /: all_labels2) { case (map, (k,v)) => map + ( k -> (v - map.getOrElse(k, 0)) )}
val acc = 100 * merged.count({ case (k, v) => v == 0 })/(all_labels2.size)

merged: scala.collection.mutable.Map[Int,Int] = Map(137 -> 0, 891 -> 0, 146 -> 14, 550 -> 0, 864 -> 0, 559 -> -9, 218 -> 0, 568 -> 0, 227 -> -38, 765 -> 0, 433 -> 0, 92 -> 0, 774 -> 0, 846 -> 0, 101 -> 0, 442 -> 0, 855 -> -20, 200 -> 0, 173 -> 29, 514 -> 0, 209 -> 0, 523 -> 0, 182 -> 0, 191 -> 0, 254 -> 36, 720 -> 0, 729 -> 0, 828 -> 0, 424 -> 30, 738 -> 1, 83 -> 0, 837 -> -30, 496 -> 0, 810 -> 0, 155 -> 0, 469 -> 0, 478 -> 0, 164 -> 0, 819 -> 0, 505 -> 0, 577 -> -9, 245 -> 0, 783 -> -12, 792 -> 20, 110 -> 0, 451 -> 0, 801 -> 0, 119 -> -3, 460 -> 21, 873 -> 11, 532 -> 0, 128 -> 1, 541 -> 0, 882 -> -15, 406 -> 0, 747 -> 0, 756 -> 0, 415 -> 0, 487 -> 0, 236 -> 0, 741 -> 0, 678 -> 0, 23 -> 0, 95 -> 0, 436 -> 0, 777 -> 0, 445 -> 0, 104 -> 0, 517 -> -20, 983 -> 0, 992 -> 0, 391 -> 0, 732 -> ...


In [143]:
println("Proportion of initial labeled nodes", 100 * last_merge.size/all_labels2.size)
println("Accuracy is",acc)

(Proportion of initial labeled nodes,29)
(Accuracy is,77)


In [144]:
//saving labels in file
m.coalesce(1).saveAsTextFile("email_labels")

org.apache.hadoop.mapred.FileAlreadyExistsException:  Output directory file:/Users/thibaultmonsel/Desktop/education/3a_cs/graphmining/projet/email_labels already exists